In [1]:
import pandas as pd
import numpy as np
import sqlite3
import matplotlib.pyplot as plt
import os
from datetime import datetime as dt
import datetime

In [2]:
#DEFINE FUNÇÕES PRINCIPAIS PARA ACESSO DO BANCO DE DADOS COM A BASE INICIAL

conn = sqlite3.connect('DIST_DB_ANEEL.db')
cursor = conn.cursor()
conn_bi = sqlite3.connect('BD_POWER_BI_FINAL.db')
cursor_bi = conn_bi.cursor()

In [3]:
#PREPARA TABELA COM INFORMAÇÕES UTEIS PARA ANALISE

tableName1 = "DIST_IND_COMPENSACAO_CONSOLIDADO"
tableName2 = "DIST_IND_DOMINIOS"
columnsNecessary = f"{tableName1}.SigAgente, {tableName1}.DscConjUndConsumidoras, {tableName1}.SigIndicador, {tableName1}.AnoIndice, {tableName1}.NumPeriodoIndice, {tableName1}.VlrIndiceEnviado, {tableName2}.DscIndicador"

query = f"""
        SELECT {columnsNecessary}
        FROM {tableName1}
        JOIN {tableName2} ON {tableName1}.SigIndicador = {tableName2}.SigIndicador
        WHERE {tableName1}.SigAgente = "ENEL CE"

        """
df = pd.read_sql_query(query,conn)



In [4]:
#AGRUPA POR ANO O TOTAL GASTO EM COMPENSAÇÃO FINANCEIRA POR NÃO ATENDIMENTO DAS NORMAS
df_analise = df.copy()
df_analise['VlrIndiceEnviado'] = df_analise['VlrIndiceEnviado'].astype(str)
df_analise['VlrIndiceEnviado'] = df_analise['VlrIndiceEnviado'].apply(lambda x: float(x.replace(",",".")))
df_analise = df_analise.groupby(['SigAgente','AnoIndice'],)['VlrIndiceEnviado'].sum()
df_analise = df_analise.reset_index()
df_analise['AumentoPercentual'] = df_analise['VlrIndiceEnviado'].pct_change()
df_analise['refDate'] = df_analise['AnoIndice']
df_analise['refDate'] = df_analise['refDate'].apply(lambda x: "01-01-"+str(x))
df_analise['refDate'] = pd.to_datetime(df_analise['refDate'])


In [5]:
#SOBE PARA O BANCO DE DADOS

df_analise.to_sql('IndCompensacaoPorAno',conn_bi, if_exists='replace', index=False)

conn.close()
conn_bi.close()